In [1]:
import time
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import torch

import optuna

sys.path.append("/jet/home/azhang19/stat 214/stat-214-lab2-group6/code/modeling")
from preprocessing import to_NCHW, pad_to_384x384, standardize_images
from classification import train_and_validate

device = "cuda" if torch.cuda.is_available() else "cpu"

torch.set_float32_matmul_precision('high')
torch.backends.cudnn.benchmark = True

use_amp = True

/jet/home/azhang19/.conda/envs/env_214/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load and preprocess data
data = np.load("/jet/home/azhang19/stat 214/stat-214-lab2-group6/data/array_data.npz")
unlabeled_images, unlabeled_masks, labeled_images, labeled_masks, labels = data["unlabeled_images"], data["unlabeled_masks"], data["labeled_images"], data["labeled_masks"], data["labels"]

unlabeled_images = pad_to_384x384(to_NCHW(unlabeled_images))
unlabeled_masks = pad_to_384x384(unlabeled_masks)

labeled_images = pad_to_384x384(to_NCHW(labeled_images))
labeled_masks = pad_to_384x384(labeled_masks)
labels = pad_to_384x384(labels)

# Convert to tensors and move to GPU
unlabeled_images = torch.tensor(unlabeled_images, dtype=torch.float32).to(device)  # [161, 8, 384, 384]
unlabeled_masks = torch.tensor(unlabeled_masks, dtype=torch.bool).to(device)    # [161, 384, 384]

labeled_images = torch.tensor(labeled_images, dtype=torch.float32).to(device)      # [3, 8, 384, 384]
labeled_masks = torch.tensor(labeled_masks, dtype=torch.bool).to(device)        # [3, 384, 384]
labels = torch.tensor(labels, dtype=torch.long).to(device)                      # [3, 384, 384]


# Standardize images
unlabeled_images, std_channel, mean_channel = standardize_images(unlabeled_images, unlabeled_masks)
labeled_images, _, _ = standardize_images(labeled_images, labeled_masks, std_channel, mean_channel)

In [3]:
import pickle
with open("/jet/home/azhang19/optuna_study_original.pkl", "rb") as f:
    study = pickle.load(f)

In [ ]:
feature = labeled_images
feature, _, _ = standardize_images(feature, labeled_masks)

num_repeat = 100
record = np.zeros((num_repeat, 2))
classifiers = []

for repeat in range(num_repeat):

    train_val_idx = [0, 1]

    # Container for metrics from each fold
    fold_records_f1 = torch.zeros(len(train_val_idx))  # For F1 (objective)
    fold_records_acc = torch.zeros(len(train_val_idx))  # For accuracy (logging)

    classifiers_in_this_repeat = []
    # Assuming feature and labels are defined globally (e.g., torch tensors)
    for i in train_val_idx:
        # Leave-one-out style split
        train_idx = [j for j in train_val_idx if j != i]
        val_idx = [i]

        # Get training and validation data
        train_data = feature[train_idx]
        train_labels = labels[train_idx]
        val_data = feature[val_idx]
        val_labels = labels[val_idx]

        # Train and validate, get both F1 and accuracy
        classifier, val_f1, val_acc = train_and_validate(
            train_data=train_data,
            train_labels=train_labels,
            val_data=val_data,
            val_labels=val_labels,
            in_channels=feature.shape[1],
            num_layers=study.best_params["num_layers"],
            kernel_size=study.best_params["kernel_size"],
            hidden_channels=study.best_params["hidden_channels"],
            epochs=study.best_params["epochs"],
            lr=study.best_params["lr"],
            weight_decay=study.best_params["weight_decay"],
            optimizer_class=torch.optim.AdamW if study.best_params['optimizer'] == 'AdamW' else torch.optim.SGD,
            loss_mix_ratio=study.best_params["loss_mix_ratio"],
            l1=study.best_params["l1"],
            class_weight=study.best_params['class_weight'],
            device=device,
            return_classifier=True,
        )
        classifiers_in_this_repeat.append((train_idx, classifier))

        fold_records_f1[i] = val_f1
        fold_records_acc[i] = val_acc

    classifiers.append(classifiers_in_this_repeat)
    avg_f1 = fold_records_f1.mean().item()
    avg_acc = fold_records_acc.mean().item()

    record[repeat] = [avg_f1, avg_acc]

record.mean(axis=0), record.std(axis=0)

W0320 01:31:57.903000 36566 site-packages/torch/_logging/_internal.py:1089] [14/0] Profiler function <class 'torch.autograd.profiler.record_function'> will be ignored
('Grad tensors ["L['self'].param_groups[0]['params'][0].grad", "L['self'].param_groups[0]['params'][1].grad", "L['self'].param_groups[0]['params'][2].grad", "L['self'].param_groups[0]['params'][3].grad", "L['self'].param_groups[0]['params'][4].grad", "L['self'].param_groups[0]['params'][5].grad", "L['self'].param_groups[0]['params'][6].grad", "L['self'].param_groups[0]['params'][7].grad", "L['self'].param_groups[0]['params'][8].grad", "L['self'].param_groups[0]['params'][9].grad"] will be copied during cudagraphs execution.If using cudagraphs and the grad tensor addresses will be the same across runs, use torch._dynamo.decorators.mark_static_address to elide this copy.',)
('Grad tensors ["L['self'].param_groups[0]['params'][0].grad", "L['self'].param_groups[0]['params'][1].grad", "L['self'].param_groups[0]['params'][2].gr

(array([0.80446373, 0.85723724]), array([0.03354775, 0.02145437]))

In [5]:
# Repeat 0, trained on 1, and validated on 0
classifiers[0][0]

([1],
 ConvHead(
   (layers): Sequential(
     (0): ConvBlock(
       (conv): Conv2d(8, 54, kernel_size=(1, 1), stride=(1, 1), padding=same)
       (bn): BatchNorm2d(54, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (activation): SiLU()
     )
     (1): ConvBlock(
       (conv): Conv2d(54, 54, kernel_size=(1, 1), stride=(1, 1), padding=same)
       (bn): BatchNorm2d(54, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (activation): SiLU()
     )
     (2): Conv2d(54, 1, kernel_size=(1, 1), stride=(1, 1), padding=same)
   )
 ))

In [6]:
# validated on 0
classifiers[0][0][1](labeled_images[0:1])

tensor([[[[ 3.5551,  3.5551,  3.5551,  ...,  1.0584,  3.5551,  3.5551],
          [ 3.5551,  3.5551,  3.5551,  ..., -0.1356,  3.5551,  3.5551],
          [ 3.5551,  3.5551,  3.5551,  ..., -3.6323,  3.5551,  3.5551],
          ...,
          [ 3.5551,  3.5551,  3.5551,  ...,  3.5551,  3.5551,  3.5551],
          [ 3.5551,  3.5551,  3.5551,  ...,  3.5551,  3.5551,  3.5551],
          [ 3.5551,  3.5551,  3.5551,  ...,  3.5551,  3.5551,  3.5551]]]],
       device='cuda:0', grad_fn=<ConvolutionBackward0>)

In [8]:
# Train on the whole dataset

classifiers = []
record = np.zeros((10, 2))

for repeat in range(10):
    classifiers_in_this_repeat = []
    # Get training and validation data
    train_data = feature[0:2]
    train_labels = labels[0:2]
    val_data = feature[[2]]
    val_labels = labels[[2]]

    # Train and validate, get both F1 and accuracy
    classifier, val_f1, val_acc = train_and_validate(
        train_data=train_data,
        train_labels=train_labels,
        val_data=val_data,
        val_labels=val_labels,
        in_channels=feature.shape[1],
        num_layers=study.best_params["num_layers"],
        kernel_size=study.best_params["kernel_size"],
        hidden_channels=study.best_params["hidden_channels"],
        epochs=study.best_params["epochs"],
        lr=study.best_params["lr"],
        weight_decay=study.best_params["weight_decay"],
        optimizer_class=torch.optim.AdamW if study.best_params['optimizer'] == 'AdamW' else torch.optim.SGD,
        loss_mix_ratio=study.best_params["loss_mix_ratio"],
        l1=study.best_params["l1"],
        class_weight=study.best_params['class_weight'],
        device=device,
        return_classifier=True,
    )
    classifiers_in_this_repeat.append((train_idx, classifier))

    record[repeat] = [val_f1.item(), val_acc.item()]

    classifiers.append(classifiers_in_this_repeat)

In [10]:
record.mean(axis=0), record.std(axis=0)

(array([0.61045697, 0.61664719]), array([0.08809232, 0.08438048]))